In [ ]:
from collections import deque, defaultdict, Counter, namedtuple

word_counts = Counter("asdoijads")

c = Counter({'a': 2, 'b': 3})

ring = deque([3,1,3], maxlen=3)

ring.append(5)

hist = defaultdict(int)

Point = namedtuple("Point", ['x', 'y'])

Point(1,3)


In [ ]:
import itertools as it
import functools as ft


count = it.count(0, 5)

x = list(it.islice(count, 5))

x

In [ ]:
def chunked(iterable, size):
    it = iter(iterable)
    while chunk := list(it.islice(it, size)):
        yield chunk

with open("demo.jsonl") as fh:
    chunked(fh)

In [ ]:
from itertools import islice
import pathlib

def chunked(iterable, size):
    """Yield successive lists (chunks) of length <= size."""
    it = iter(iterable)  # always use the real built-in
    while (chunk := list(islice(it, size))):
        yield chunk

path = pathlib.Path('demo.jsonl')

with path.open() as fh:
    for batch in chunked(fh, 5):
        pass


In [ ]:
c = it.count(0, 5)

In [ ]:
chunk = list(it.islice(c, 5))
chunk

In [ ]:
import copy

orig = [[1, 2], [3, 4]]
shallow = copy.copy(orig)

shallow[0] is orig[0]

In [94]:
from collections import Counter
example = [
    {"visit_id": "v1", "doctor_id": "dr_42",
     "note": "Patient stable. Continue meds."},
    {"visit_id": "v2", "doctor_id": "dr_99",
     "note": "↑ BP, schedule follow-up in 2 weeks."},
    {"visit_id": "v3", "doctor_id": "dr_42",
     "note": "No complications; discharge tomorrow."},
]

tokens = len(example[0]["note"].split())
tokens

4

In [105]:
from dataclasses import dataclass

@dataclass
class VisitStats:
    doctor_id: str
    num_visits: int
    total_tokens: int      
    avg_tokens_per_visit: float

def summarise_visits(visits: list[dict[str, str]]) -> list[VisitStats]:
    docs = {}
    for visit in visits:
        tokens = len(visit["note"].split())
        if visit["doctor_id"] in docs:
            docs[visit["doctor_id"]].num_visits += 1
            docs[visit["doctor_id"]].total_tokens += tokens
        else:
            docs[visit["doctor_id"]] = VisitStats(
                doctor_id=visit["doctor_id"],
                num_visits = 1,
                total_tokens=tokens,
                avg_tokens_per_visit=0
            )
            

    visitstats = list(docs.values())
    for vs in visitstats:
        vs.avg_tokens_per_visit = vs.total_tokens / vs.num_visits
    
    return sorted(
        visitstats, key=lambda a: (-a.total_tokens, a.doctor_id)
    )


example = [
    {"visit_id": "v1", "doctor_id": "dr_42",
     "note": "Patient stable. Continue meds."},
    {"visit_id": "v2", "doctor_id": "dr_99",
     "note": "↑ BP, schedule follow-up in 2 weeks."},
    {"visit_id": "v3", "doctor_id": "dr_42",
     "note": "No complications; discharge tomorrow."},
]

summarise_visits(example)


[VisitStats(doctor_id='dr_42', num_visits=2, total_tokens=8, avg_tokens_per_visit=4.0),
 VisitStats(doctor_id='dr_99', num_visits=1, total_tokens=7, avg_tokens_per_visit=7.0)]

In [ ]:
def heightChecker(heights) -> int:
        # expected: List[int] of heights ascending order where expected[i] is the
        # height of student i.
        expected = sorted(heights) # o(n logn)
        diff = 0
        for i in range(len(expected)):
            if expected[i] != heights[i]:
                  diff += 1
        return diff

heights = [1,2,3,4,5]
heightChecker(heights)

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

In [ ]:
from experiments.ngpt.n_transformer import nTransformer, nConfig

model_config = nConfig(
    vocab_size=50258,
    seq_len=512,
    d_model=768,
    num_layers=8,
    num_heads=8,
    num_kv_heads=8,
    mlp_bias=False,
    mlp_dropout=0.0,
    attention_bias=False,
    attention_dropout=0.0,
    weight_tying=False,
)

model = nTransformer(model_config)

In [ ]:
from omni.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

In [ ]:
def causal_attention_mask(sequence_length: int, dtype=torch.float32):
    mask = torch.triu(torch.ones((1, 1, sequence_length, sequence_length), dtype=dtype), diagonal=1)
    mask = mask.masked_fill(mask == 1, float("-inf")) 
    return mask

mask = causal_attention_mask(5)

In [ ]:
mask

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("data/pretokenized_fineweb-edu-2BT")

In [ ]:
tokenizer.pad_token_id

In [ ]:
(x == 50257).sum()

In [ ]:
for i in range(0, 15):

    print(tokenizer.decode(ds[i]["input_ids"]))

In [ ]:
import torch

from omni.architectures.llama import LlamaConfig
from omni.modules.transformer import Transformer
from omni.preprocessing.tokenizer import AutoTokenizer
from omni.utils.system import auto_device

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

llama_config = LlamaConfig(
    vocab_size=50258,
    seq_len=512,
    d_model=256,
    num_heads=8,
    num_kv_heads=8,
    num_layers=4,
    rope_theta=0.1,
    norm_eps=1e-6,
    activation_fn="silu",
    mlp_bias=False,
    mlp_dropout=0.0,
    attention_bias=False,
    attention_dropout=0.0,
    pos_encoding_type="rope",
    mlp="mlp_swiglu",
    normalization="rmsnorm",
    attention="gqa",
)

model = Transformer(llama_config)

## create KV cache

checkpoint = torch.load("checkpoints/llama-30M_20250123_104138/init.ckpt", weights_only=True)
model.load_state_dict(checkpoint["model"])

device="mps"
model = model.to(device)
model.eval()

In [ ]:
device="mps"
prompt = "Once upon a time"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

In [ ]:
outputs = model(input_ids)
next_token_logits = outputs[:, -1, :]

In [ ]:
top_k_values, top_k_indices = torch.topk(logits, self.top_k, dim=-1)

In [ ]:
top_k_values = torch.tensor([float("-inf"), 1.0, 2.0])

top_k_values
nn.functional.softmax(top_k_values, dim=-1)

In [ ]:
float("-inf")

In [ ]:
sorted_logits, sorted_indices = torch.sort(
    top_k_values, dim=-1, descending=True
)
cumulative_probs = torch.cumsum(
    nn.functional.softmax(sorted_logits, dim=-1), dim=-1
)

In [ ]:
next_token_logits

In [ ]:
visualize_logit_distribution(next_token_logits[0], top_k=500)

In [ ]:
import torch
import matplotlib.pyplot as plt

def visualize_logit_distribution(logits, top_k=500):
    """
    Visualizes the logit distribution by focusing on the top-k logits.

    Args:
        logits (torch.Tensor): Logits tensor of shape (vocab_size,).
        top_k (int): Number of top logits to visualize. Defaults to 500.
    """
    if logits.dim() != 1:
        raise ValueError("Logits tensor must be 1-dimensional (vocab_size,).")

    # Convert logits to probabilities and take log
    log_probs = torch.log_softmax(logits, dim=0)

    # Get the top-k log probabilities and their indices
    top_log_probs, top_indices = torch.topk(log_probs, k=top_k)

    # Sort the top-k log probabilities for visualization
    sorted_log_probs, sorted_indices = torch.sort(top_log_probs, descending=True)

    # Plot the distribution
    plt.figure(figsize=(10, 6))
    plt.plot(sorted_log_probs.detach().cpu().numpy(), marker="o", linestyle="-")
    plt.title(f"Log Probability Distribution (Top {top_k})", fontsize=14)
    plt.xlabel("Rank", fontsize=12)
    plt.ylabel("Log Probability Value", fontsize=12)
    plt.grid(alpha=0.5)
    plt.show()


In [ ]:
import tokenizers.processors as processors

def _add_bos_token(tokenizer: AutoTokenizer):
    tokenizer._tokenizer.post_processor = processors.Sequence(
        [
            processors.ByteLevel(trim_offsets=False),
            processors.TemplateProcessing(
                single=f"{tokenizer.bos_token}:0 $A:0",
                pair=f"{tokenizer.bos_token}:0 $A:0 {tokenizer.bos_token}:1 $B:1",
                special_tokens=[
                    (tokenizer.bos_token, tokenizer.bos_token_id),
                ],
            ),
        ]
    )
    return tokenizer

In [ ]:
tokenizer2 = _add_bos_token(tokenizer) 

In [ ]:
torch.log(torch.tensor(50000))

In [ ]:
test = "thjis is a test, this is a test"

output = tokenizer2(test, padding="max_length", max_length=3, truncation=True, return_overflowing_tokens=True)

In [ ]:
d_model=128
hidden_dim = 4 * int(2 * d_model / 3)
hidden_dim = 4 * (
    (hidden_dim + 4 - 1) // 4
)

hidden_dim

In [ ]:
import torch

logits = torch.tensor([1,5,10], dtype=torch.float32)
torch.softmax(logits, dim=0)

In [ ]:
tokenizer.special_tokens_map

In [ ]:
tokenizer.encode(x, padding="max_length", max_length=max_length, truncation=True, return_overflowing_tokens=True)

In [ ]:
tokenizer(test, )

In [ ]:
output

In [ ]:
from omni.modules.pos_embeddings import precompute_freqs_cis_real

pos_embeddings = precompute_freqs_cis_real(64, 512)

In [ ]:
def _tokenize(dataset, tokenizer: AutoTokenizer, num_proc):
    print("Tokenizing dataset...")
    dataset = dataset.map(
        lambda x: tokenizer(
            x["text"],
            truncation=True,
            max_length=512,
            padding="max_length",
            return_overflowing_tokens=True,
        ),
        batched=True,
        num_proc=num_proc,
        remove_columns=dataset.column_names,
    )
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return dataset

In [23]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    """
    A bare-bones implementation of a Multi-Head Attention module
    as described in Section 2.1 of arXiv:2505.06708v1.
    """
    def __init__(self, d_model: int, num_heads: int, config = None):
        """
        Args:
            d_model (int): The total dimensionality of the model.
            num_heads (int): The number of parallel attention heads.
        """
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # Stage 1: QKV Linear Projections 
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)

        # Gate
        self.W_q_gate = nn.Linear(d_model, d_model, bias=False)
        self.W_k_gate = nn.Linear(d_model, d_model, bias=False)
        self.W_v_gate = nn.Linear(d_model, d_model, bias=False)

        # headwise score
        self.W_q_headwise_gate = nn.Linear(d_model, num_heads, bias=False)
        self.W_k_headwise_gate = nn.Linear(d_model, num_heads, bias=False)
        self.W_v_headwise_gate = nn.Linear(d_model, num_heads, bias=False)
        
        # head shared 
        self.W_q_head_shared_gate = nn.Linear(d_model, self.head_dim, bias=False)
        self.W_k_head_shared_gate = nn.Linear(d_model, self.head_dim, bias=False)
        self.W_v_head_shared_gate = nn.Linear(d_model, self.head_dim, bias=False)

        # Stage 4: Final Output Layer 
        self.W_o = nn.Linear(d_model, d_model, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_len, d_model)
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, seq_len, d_model)
        """
        batch_size, seq_len, _ = x.shape

        # --- Stage 1: QKV Linear Projections ---
        # Project input `x` into Q, K, V tensors.
        # Shape: (batch_size, seq_len, d_model)
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)

        # gating QKV projections
        q = q * torch.sigmoid(self.W_q_gate(x)) 
        k = k * torch.sigmoid(self.W_v_gate(x))  
        v = v * torch.sigmoid(self.W_v_gate(x)) 

        # --- Stage 3: Multi-Head Splitting ---
        # Reshape Q, K, V for multi-head processing.
        # New Shape: (batch_size, num_heads, seq_len, head_dim)
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # headwise score
        q = q * torch.sigmoid(self.W_q_headwise_gate(x)).view(batch_size, self.num_heads, seq_len, 1)
        k = k * torch.sigmoid(self.W_k_headwise_gate(x)).view(batch_size, self.num_heads, seq_len, 1)
        v = v * torch.sigmoid(self.W_k_headwise_gate(x)).view(batch_size, self.num_heads, seq_len, 1)

        # head shared QKV gating
        q = q * torch.sigmoid(self.W_q_head_shared_gate(x)).unsqueeze(1)
        k = k * torch.sigmoid(self.W_k_head_shared_gate(x)).unsqueeze(1)
        v = v * torch.sigmoid(self.W_k_head_shared_gate(x)).unsqueeze(1)

        # --- Stage 2: Scaled Dot-Product Attention (SDPA) ---
        # Shape of attn_scores: (batch_size, num_heads, seq_len, seq_len)
        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Apply softmax to get attention weights.
        attn_weights = torch.softmax(attn_scores, dim=-1)
        
        # Multiply weights by V to get the attention output.
        # Shape: (batch_size, num_heads, seq_len, head_dim)
        attention_output = attn_weights @ v

        # --- Stage 3: Multi-Head Concatenation ---
        # Reshape the attention output to concatenate heads.
        # transpose(1, 2) -> (batch_size, seq_len, num_heads, head_dim)
        # contiguous().view() -> (batch_size, seq_len, d_model)
        concatenated_output = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)

        # --- Stage 4: Final Output Projection ---
        # Pass the concatenated output through the final linear layer.
        # Shape: (batch_size, seq_len, d_model)
        final_output = self.W_o(concatenated_output)

        return final_output

In [ ]:
def forward(
        self,
        x: Float[Tensor, "batch seq d_model"],
        mask: Float[Tensor, "1 1 seq seq"],
        pos_info: Optional[Tensor],
        kv_cache,
    ):
        batch_size, seq_length, d_model = x.size()

        qkv = self.W_QKV(x)
        q, k, v = qkv.chunk(3, dim=-1)

        q = q.reshape(batch_size, seq_length, self.n_heads, self.head_dim)
        k = k.reshape(batch_size, seq_length, self.n_heads, self.head_dim)
        v = v.reshape(batch_size, seq_length, self.n_heads, self.head_dim)

        # Apply gating mechanisms using direct config access.
        if self.config.qkv_gate_enabled:
            q = q * torch.sigmoid(self.W_q_gate(x)) 
            k = k * torch.sigmoid(self.W_v_gate(x))  
            v = v * torch.sigmoid(self.W_v_gate(x)) 

In [ ]:
batch_size = 1     # Number of sequences to process in parallel
seq_len = 64     # The length of each input sequence
d_model = 512    # The dimensionality of the model's embeddings
num_heads = 4      # Number of attention heads (512 is divisible by 8)

mha_baseline = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

dummy_x = torch.randn(batch_size, seq_len, d_model)

output = mha_baseline(dummy_x)

In [ ]:
from typing import Literal, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from jaxtyping import Complex, Float, Int
from torch import Tensor

from omni.modules.pos_embeddings import apply_rope_real

AttentionType = Literal["mha", "gqa"]

def forward(
        self,
        x: Float[Tensor, "batch seq d_model"],
        mask: Float[Tensor, "1 1 seq seq"],
        pos_info: Optional[Tensor],
        kv_cache,
    ):
        # ...
    
        output: Float[Tensor, "batch num_heads seq head_dim"] = F.scaled_dot_product_attention(
            q, 
            k, 
            v, 
            attn_mask=mask, 
            dropout_p=self.attn_dropout.p if self.training else 0.0
        )
        
        output = output.transpose(1, 2)
        if self.config.attention_output_gate_enabled:
            # self.W_attn_out_gate = nn.Linear(d_model, d_model)
            gate_scores = torch.sigmoid(self.W_attn_out_gate(x))
            output = output * gate_scores.reshape(batch_size, seq_len, self.n_heads, self.head_dim)
        
        # ...

        return output

In [5]:
output.shape

torch.Size([1, 64, 512])